2.1 Tipos de data e limpeza de orders

In [0]:
from pyspark.sql import functions as F

orders = (spark.table("bronze_olist_orders")
  .withColumn("order_purchase_timestamp", F.to_timestamp("order_purchase_timestamp"))
  .withColumn("order_approved_at", F.to_timestamp("order_approved_at"))
  .withColumn("order_delivered_carrier_date", F.to_timestamp("order_delivered_carrier_date"))
  .withColumn("order_delivered_customer_date", F.to_timestamp("order_delivered_customer_date"))
  .withColumn("order_estimated_delivery_date", F.to_timestamp("order_estimated_delivery_date"))
)

orders.write.format("delta").mode("overwrite").saveAsTable("silver_olist_orders")


2.2 Produtos com tradução de categoria


In [0]:
products = spark.table("bronze_olist_products").drop("ingested_at")
trans = spark.table("bronze_olist_translation").drop("ingested_at")

products2 = products.join(trans, on="product_category_name", how="left")
products2.write.format("delta").mode("overwrite").saveAsTable("silver_olist_products")


2.3 Itens enriquecidos com métricas de entrega/atraso

In [0]:
from pyspark.sql import functions as F

items = spark.table("bronze_olist_order_items").drop("ingested_at")
orders = spark.table("silver_olist_orders").drop("ingested_at")

items_enriched = (items.join(orders, "order_id", "left")
  .withColumn("entrega_dias",
      F.datediff(F.to_date("order_delivered_customer_date"), F.to_date("order_purchase_timestamp")))
  .withColumn("prazo_estimado_dias",
      F.datediff(F.to_date("order_estimated_delivery_date"), F.to_date("order_purchase_timestamp")))
  .withColumn("atraso_dias",
      F.datediff(F.to_date("order_delivered_customer_date"), F.to_date("order_estimated_delivery_date")))
)

items_enriched.write.format("delta").mode("overwrite").saveAsTable("silver_olist_order_items_enriched")
